In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [15]:
# Exemplo de base simulada
dados = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4],
    'tipo_cliente': ['correntista', 'nao_correntista', 'cartonista', 'nao_correntista'],
    'parcela_em_aberto': [1, 1, 0, 1],
    'modalidade_pagamento': ['debito', 'carne', 'carne', 'debito'],
    'faixa_atraso': ['em dia', '1-15 dias', '60+ dias', '16-60 dias'],
    'tempo_contrato_meses': [18, 3, 36, 6],
    'acionou_canal_para_servico_digital': [0, 1, 1, 0],
    'qtd_acessos_app': [10, 0, 4, 0],
    'nivel_app': [2, 0, 1, 0],
    'uso_whatsapp': [5, 12, 3, 7],
    'diversidade_servicos': [3, 2, 1, 4],
    'taxa_resolucao_digital': [0.85, 0.6, 0.4, 0.9],
    'dias_ultima_interacao': [5, 10, 40, 2]
})

In [16]:
dados.head()

,id_cliente,tipo_cliente,parcela_em_aberto,modalidade_pagamento,faixa_atraso,tempo_contrato_meses,acionou_canal_para_servico_digital,qtd_acessos_app,nivel_app,uso_whatsapp,diversidade_servicos,taxa_resolucao_digital,dias_ultima_interacao
0,1,correntista,1,debito,em dia,18,0,10,2,5,3,0.85,5
1,2,nao_correntista,1,carne,1-15 dias,3,1,0,0,12,2,0.60,10
2,3,cartonista,0,carne,60+ dias,36,1,4,1,3,1,0.40,40
3,4,nao_correntista,1,debito,16-60 dias,6,0,0,0,7,4,0.90,2


In [17]:
# 1. Engenharia de variáveis
dados['recencia'] = dados['dias_ultima_interacao'].apply(lambda x: 1 / (1 + x))
dados['pagamento_debito'] = dados['modalidade_pagamento'].apply(lambda x: 1 if x == 'debito' else 0)

In [19]:
mapa_atraso = {'em dia': 0, '1-15 dias': 1, '16-60 dias': 2, '60+ dias': 3}
dados['faixa_atraso'] = dados['faixa_atraso'].map(mapa_atraso)

In [21]:
dados['penalizacao_acionamento'] = dados['acionou_canal_para_servico_digital'].apply(lambda x: -0.1 if x == 1 else 0)

In [23]:
# 2. Normalização
variaveis_para_normalizar = [
    'qtd_acessos_app', 'nivel_app', 'uso_whatsapp', 'diversidade_servicos',
    'taxa_resolucao_digital', 'recencia', 'tempo_contrato_meses'
]

In [24]:
scaler = MinMaxScaler()
normalizadas = pd.DataFrame(
    scaler.fit_transform(dados[variaveis_para_normalizar]),
    columns=[f"{col}_norm" for col in variaveis_para_normalizar]
)

In [ ]:
dados = pd.concat([dados, normalizadas], axis=1)

In [26]:
# 3. Score digital com lógica contextual
def calcular_score(linha):
    if linha['parcela_em_aberto'] == 0:
        return None  # Cliente sem expectativa de uso no mês

    if linha['tipo_cliente'] == 'nao_correntista':
        score = (
            0.6 * linha['uso_whatsapp_norm'] +
            0.4 * linha['taxa_resolucao_digital_norm']
        )
    else:
        score = (
            0.15 * linha['qtd_acessos_app_norm'] +
            0.15 * linha['nivel_app_norm'] +
            0.15 * linha['uso_whatsapp_norm'] +
            0.15 * linha['diversidade_servicos_norm'] +
            0.10 * linha['taxa_resolucao_digital_norm'] +
            0.10 * linha['recencia_norm'] +
            0.10 * linha['tempo_contrato_meses_norm'] +
            0.05 * linha['pagamento_debito']  # binária, já em 0 ou 1
        )

    return max(0, min(1, score + linha['penalizacao_acionamento']))  # score entre 0 e 1

In [27]:
dados['score_digital'] = dados.apply(calcular_score, axis=1)

In [ ]:

# 4. Classificação
def classificar(score):
    if pd.isnull(score):
        return 'Sem necessidade no mês'
    elif score >= 0.7:
        return 'Alto'
    elif score >= 0.4:
        return 'Médio'
    else:
        return 'Baixo'

dados['nivel_digitalizacao'] = dados['score_digital'].apply(classificar)

In [30]:
dados

,id_cliente,tipo_cliente,parcela_em_aberto,modalidade_pagamento,faixa_atraso,tempo_contrato_meses,acionou_canal_para_servico_digital,qtd_acessos_app,nivel_app,uso_whatsapp,...,penalizacao_acionamento,qtd_acessos_app_norm,nivel_app_norm,uso_whatsapp_norm,diversidade_servicos_norm,taxa_resolucao_digital_norm,recencia_norm,tempo_contrato_meses_norm,score_digital,nivel_digitalizacao
0,1,correntista,1,debito,0,18,0,10,2,5,...,0.0,1.0,1.0,0.222222,0.666667,0.9,0.460526,0.454545,0.664841,Médio
1,2,nao_correntista,1,carne,1,3,1,0,0,12,...,-0.1,0.0,0.0,1.000000,0.333333,0.4,0.215311,0.000000,0.660000,Médio
2,3,cartonista,0,carne,3,36,1,4,1,3,...,-0.1,0.4,0.5,0.000000,0.000000,0.0,0.000000,1.000000,NaN,Sem necessidade no mês
3,4,nao_correntista,1,debito,2,6,0,0,0,7,...,0.0,0.0,0.0,0.444444,1.000000,1.0,1.000000,0.090909,0.666667,Médio


In [29]:
# 5. Resultado
print(dados[[
    'id_cliente', 'tipo_cliente', 'parcela_em_aberto', 'score_digital',
    'nivel_digitalizacao', 'modalidade_pagamento', 'faixa_atraso',
    'tempo_contrato_meses', 'acionou_canal_para_servico_digital'
]])

   id_cliente     tipo_cliente  parcela_em_aberto  score_digital  \
0           1      correntista                  1       0.664841   
1           2  nao_correntista                  1       0.660000   
2           3       cartonista                  0            NaN   
3           4  nao_correntista                  1       0.666667   

      nivel_digitalizacao modalidade_pagamento  faixa_atraso  \
0                   Médio               debito             0   
1                   Médio                carne             1   
2  Sem necessidade no mês                carne             3   
3                   Médio               debito             2   

   tempo_contrato_meses  acionou_canal_para_servico_digital  
0                    18                                   0  
1                     3                                   1  
2                    36                                   1  
3                     6                                   0  


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Exemplo de base
dados = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4],
    'tipo_cliente': ['correntista', 'nao_correntista', 'cartonista', 'nao_correntista'],
    'parcela_em_aberto': [1, 1, 0, 1],  # 1 = sim, 0 = não
    'qtd_acessos_app': [10, 0, 4, 0],
    'nivel_app': [2, 0, 1, 0],
    'uso_whatsapp': [5, 12, 3, 7],
    'diversidade_servicos': [3, 2, 1, 4],
    'taxa_resolucao_digital': [0.85, 0.6, 0.4, 0.9],
    'dias_ultima_interacao': [5, 10, 40, 2]
})

# Recência invertida
dados['recencia'] = dados['dias_ultima_interacao'].apply(lambda x: 1 / (1 + x))

# Normalização
variaveis = ['qtd_acessos_app', 'nivel_app', 'uso_whatsapp', 'diversidade_servicos', 'taxa_resolucao_digital', 'recencia']
scaler = MinMaxScaler()
normalizado = pd.DataFrame(scaler.fit_transform(dados[variaveis]), columns=[f"{v}_norm" for v in variaveis])

# Junta com os dados originais
dados = pd.concat([dados, normalizado], axis=1)

# Score ajustado conforme perfil
def calcular_score(linha):
    if linha['parcela_em_aberto'] == 0:
        return None  # Não é justo avaliar quem não tinha motivo pra usar o canal

    if linha['tipo_cliente'] == 'nao_correntista':
        return (
            0.6 * linha['uso_whatsapp_norm'] +
            0.4 * linha['taxa_resolucao_digital_norm']
        )
    else:
        return (
            0.2 * linha['qtd_acessos_app_norm'] +
            0.2 * linha['nivel_app_norm'] +
            0.2 * linha['uso_whatsapp_norm'] +
            0.2 * linha['diversidade_servicos_norm'] +
            0.1 * linha['taxa_resolucao_digital_norm'] +
            0.1 * linha['recencia_norm']
        )

dados['score_digital'] = dados.apply(calcular_score, axis=1)

# Classificação
def classificar(score):
    if pd.isnull(score):
        return 'Sem necessidade no mês'
    elif score >= 0.7:
        return 'Alto'
    elif score >= 0.4:
        return 'Médio'
    else:
        return 'Baixo'

dados['nivel_digitalizacao'] = dados['score_digital'].apply(classificar)

# Exibe resultado
print(dados[['id_cliente', 'tipo_cliente', 'score_digital', 'nivel_digitalizacao']])